# EXTERNAL DEPENDENCIES OF THIS WORKFLOW:

This workflow invokes "owltools" (https://github.com/owlcollab/owltools/wiki/Install-OWLTools)
which is a Java language tool. This needs has to be installed on the PATH visible to the Notebook.
Thus, if necessary, we install it here. This build requires the *maven* build tool which 
we assume already installed on the system under the directory ```/opt/maven/current/bin```


In [12]:
! cd /opt; test -d /opt/owltools || git clone https://github.com/owlcollab/owltools.git 

Cloning into 'owltools'...
remote: Enumerating objects: 42516, done.
remote: Total 42516 (delta 0), reused 0 (delta 0), pack-reused 42516
Receiving objects: 100% (42516/42516), 73.11 MiB | 2.97 MiB/s, done.
Resolving deltas: 100% (26071/26071), done.


## Maven Build Warning
A Maven build warning for the OWLTools-Runner mentions that the **build.directory** environment variable is deprecated and recommends that the **project.build.directory** variable be used instead _(RMB: 17/2/2019 -  a pull request issued to the project in Github)_

In [1]:
! export PATH=/opt/maven/current/bin:$PATH; cd /opt/owltools/OWLTools-Parent ; mvn clean install

[INFO] Scanning for projects...
[INFO] ------------------------------------------------------------------------
[INFO] Reactor Build Order:
[INFO] 
[INFO] OWLTools-Parent                                                    [pom]
[INFO] OWLTools-Core                                                      [jar]
[INFO] OWLTools-Annotation                                                [jar]
[INFO] OWLTools-Oort                                                      [jar]
[INFO] OWLTools-Sim                                                       [jar]
[INFO] OWLTools-Web                                                       [jar]
[INFO] Lego                                                               [jar]
[INFO] OWLTools-Solr                                                      [jar]
[INFO] OWLTools-Runner                                                    [jar]
[INFO] OWLTools-NCBI                                                      [jar]
[INFO] Golr-Client                                  

# Install Python Dependencies (including NCATS MVP Module Library)

In [1]:
# You need to figure out where your Python site-packages are...
# After getting the answer to the following Jupyter 'type' directive
# change the path component 'bin' to 'lib' path, then append the suffix "/site-packages"
# Then use thge resultng path to find the location at which you can complete 'Step 1' in the following cell.
!type python

python is /anaconda3/bin/python


In [2]:
'''
Assuming that we've checked out the NCATS mvp-module-library 
alongside our ncats-translator-workflows project, this should work
'''
import sys
sys.path.append("../../mvp-module-library")
#
# Hack to get around problematic updating of distutils installed PyYAML and a slightly older pandas requiring a compatible numpy
#
# Step 1 - MANUALLY remove the PyYAML and numpy module directories from the your site-packages noted above, then...
#
# Step 2 - Install your system (required new PyYAML updated version should be installed alongside?)#
!{sys.executable} -m pip install -r requirements.txt

  Using cached https://files.pythonhosted.org/packages/46/e4/4a0cc770e4bfb34b4e10843805fef67b9a94027e59162a586c776f35c5bb/numpy-1.16.1-cp37-cp37m-macosx_10_6_intel.macosx_10_9_intel.macosx_10_9_x86_64.macosx_10_10_intel.macosx_10_10_x86_64.whl


In [3]:
from BioLink.biolink_client import BioLinkWrapper
import pandas as pd

# Workflow II Rare Disease Candidates

In [4]:
from Modules.Mod0_lookups import LookUp

# workflow input is a disease identifier
lu = LookUp()

Mod O DiseaseGeneLookup metadata:
{'data_type': 'disease',
 'input_type': {'complexity': 'single', 'id_type': ['MONDO', 'DO', 'OMIM']},
 'limit': None,
 'output_type': {'complexity': 'set', 'id_type': 'HGNC'},
 'predicate': 'blm:gene associated with condition',
 'source': 'Monarch Biolink',
 'taxon': 'human'}


In [5]:
input_disease = 'MONDO:0008667' # Von Hippel Lindau (VHL)
input_object = {
    'input': input_disease,
    'parameters': {
        'taxon': 'human',
        'threshold': None,
    },
}

lu.load_input_object(input_object=input_object)
input_object = lu.input_object

{'description': 'Von Hippel-Lindau disease (VHL) is a familial cancer '
                'predisposition syndrome associated with a variety of '
                'malignant and benign neoplasms, most frequently retinal, '
                'cerebellar, and spinal hemangioblastoma, renal cell carcinoma '
                '(RCC), and pheochromocytoma.',
 'id': 'MONDO:0008667',
 'label': 'von Hippel-Lindau disease'}


In [6]:
# get genes associated with disease from Biolink
disease_associated_genes = lu.disease_geneset_lookup()
# create list of gene curies for downstream module input
input_curie_set = disease_associated_genes[['hit_id', 'hit_symbol']].to_dict(orient='records')
# # show the 
disease_associated_genes['modules'] = 'Mod0'
disease_associated_genes
# genes to investigate TNF, FANCD2OS

,input_id,input_symbol,hit_id,hit_symbol,relation,sources,modules
0,MONDO:0008667,von Hippel-Lindau disease,HGNC:12687,VHL,pathogenic_for_condition,"ctd, omim, orphane, clinvar",Mod0
1,MONDO:0008667,von Hippel-Lindau disease,HGNC:1582,CCND1,contributes to,"omim, ctd",Mod0
2,MONDO:0008667,von Hippel-Lindau disease,HGNC:23057,BRK1,pathogenic_for_condition,clinvar,Mod0


# Mod1A Functional Similarity
## Find similar genes based on GO functional annotations using OntoBio Jaccard similarity

In [7]:
from Modules.Mod1A_functional_sim import FunctionalSimilarity

## Mod1A_human

In [8]:
# Module specification
mod1a_input_object_human = {
    'input': input_curie_set,
    'parameters': {
        'taxon': 'human',
        'threshold': 0.75,  # jaccard index threshold
    },
}
    
func_sim_human = FunctionalSimilarity()

Mod1A Functional Similarity metadata:
{'input_type': {'complexity': 'set', 'data_type': 'gene', 'id_type': 'HGNC'},
 'output_type': {'complexity': 'set', 'data_type': 'gene', 'id_type': 'HGNC'},
 'predicate': ['blm:macromolecular machine to biological process association',
               'macromolecular machine to molecular activity association'],
 'source': 'Monarch Biolink'}


In [9]:
func_sim_human.load_input_object(mod1a_input_object_human)
func_sim_human.load_gene_set() 

In [10]:
func_sim_human.load_associations()

FileNotFoundError: [Errno 2] No such file or directory: 'owltools': 'owltools'

In [11]:
mod1a_results = func_sim_human.compute_similarity()

In [10]:
Mod1A_results_human = pd.DataFrame(mod1a_results)
Mod1A_results_human = Mod1A_results_human[~Mod1A_results_human['hit_id'].isin(disease_associated_genes['hit_id'].tolist())].sort_values('score', ascending=False)
Mod1A_results_human['module'] = 'Mod1A'
Mod1A_results_human

,hit_id,hit_symbol,input_id,input_symbol,score,module
17,HGNC:21700,RAD9B,HGNC:3583,FANCB,0.920000,Mod1A
15,HGNC:26171,FAAP100,HGNC:3583,FANCB,0.857143,Mod1A
22,HGNC:24994,INIP,HGNC:3583,FANCB,0.851852,Mod1A
26,HGNC:21700,RAD9B,HGNC:3586,FANCE,0.840000,Mod1A
20,HGNC:28467,FAAP24,HGNC:3583,FANCB,0.827586,Mod1A
16,HGNC:25021,ASTE1,HGNC:3583,FANCB,0.793103,Mod1A
18,HGNC:22223,EEPD1,HGNC:3583,FANCB,0.785714,Mod1A
25,HGNC:26171,FAAP100,HGNC:3586,FANCE,0.785714,Mod1A
8,HGNC:20922,SLX1A,HGNC:23845,SLX4,0.782313,Mod1A
27,HGNC:22223,EEPD1,HGNC:3586,FANCE,0.777778,Mod1A


# MOD1B Phenotype Similarity
## Find similar genes based on OwlSim calculated Phenotype Similarity

## Mod1B Human

In [12]:
from Modules.Mod1B1_phenotype_similarity import PhenotypeSimilarity

In [13]:
# Module specification
mod1b_input_object_human = {
    'input': input_curie_set,
     'parameters': {
        'taxon': 'human',
        'threshold': .50,
    },
}


pheno_sim_human = PhenotypeSimilarity()

Mod1B Phenotype Similarity metadata:
{'input_type': {'complexity': 'set', 'data_type': 'gene', 'id_type': 'HGNC'},
 'output_type': {'complexity': 'set', 'data_type': 'gene', 'id_type': 'HGNC'},
 'predicate': ['blm:has phenotype'],
 'source': 'Monarch Biolink'}


In [14]:
pheno_sim_human.load_input_object(mod1b_input_object_human)
pheno_sim_human.load_gene_set()

In [15]:
pheno_sim_human.load_associations()

ERROR:pysolr:Failed to connect to server at 'https://solr.monarchinitiative.org/solr/golr/select/?q=%2A%3A%2A&start=100000&fq=subject_category%3A%22gene%22&fq=object_category%3A%22phenotype%22&fq=subject_taxon_closure%3A%22NCBITaxon%3A9606%22&facet=off&facet.limit=0&facet.mincount=1&fl=subject%2Csubject_label%2Crelation%2Cobject%2Creference%2Cqualifier%2Cis_redundant_for%2Ctype%2Cevidence%2Cevidence_label%2Cevidence_type%2Cevidence_type_label%2Cevidence_with%2Cevidence_closure%2Cevidence_closure_label%2Cevidence_subset_closure%2Cevidence_subset_closure_label%2Cevidence_type_closure%2Cevidence_type_closure_label&rows=100000&wt=json', are you sure that URL is correct? Checking it in a browser might help: HTTPSConnectionPool(host='solr.monarchinitiative.org', port=443): Read timed out.
Traceback (most recent call last):
  File "/Users/tputman/.virtualenvs/modules/lib/python3.6/site-packages/urllib3/response.py", line 331, in _error_catcher
    yield
  File "/Users/tputman/.virtualenvs/mod

SolrError: Failed to connect to server at 'https://solr.monarchinitiative.org/solr/golr/select/?q=%2A%3A%2A&start=100000&fq=subject_category%3A%22gene%22&fq=object_category%3A%22phenotype%22&fq=subject_taxon_closure%3A%22NCBITaxon%3A9606%22&facet=off&facet.limit=0&facet.mincount=1&fl=subject%2Csubject_label%2Crelation%2Cobject%2Creference%2Cqualifier%2Cis_redundant_for%2Ctype%2Cevidence%2Cevidence_label%2Cevidence_type%2Cevidence_type_label%2Cevidence_with%2Cevidence_closure%2Cevidence_closure_label%2Cevidence_subset_closure%2Cevidence_subset_closure_label%2Cevidence_type_closure%2Cevidence_type_closure_label&rows=100000&wt=json', are you sure that URL is correct? Checking it in a browser might help: HTTPSConnectionPool(host='solr.monarchinitiative.org', port=443): Read timed out.

In [ ]:
mod1b_results = pheno_sim_human.compute_similarity()

In [ ]:
# compute phenotype similarity
Mod1B_results = pd.DataFrame(mod1b_results)
Mod1B_results = Mod1B_results[~Mod1B_results['hit_id'].isin(disease_associated_genes['hit_id'].tolist())].sort_values('score', ascending=False)
Mod1B_results['module'] = 'Mod1B'
Mod1B_results

# Mod1E Protein Interaction

## Mod1E Human

In [ ]:
from Modules.Mod1E_interactions import GeneInteractions

In [ ]:
# Module specification
mod1E_input_object_human = {
    'input': input_curie_set,
     'parameters': {
        'taxon': 'human',
        'threshold': None,
    },
}


interactions_human = GeneInteractions()

In [ ]:
interactions_human.load_input_object(mod1E_input_object_human)
interactions_human.load_gene_set()

In [ ]:
mod1e_results = interactions_human.get_interactions()

In [ ]:
Mod1E_results_human = pd.DataFrame(mod1e_results)

In [ ]:
counts = Mod1E_results_human['hit_symbol'].value_counts().rename_axis('unique_values').to_frame('counts').reset_index()
high_counts = counts[counts['counts'] > 12]['unique_values'].tolist()

In [ ]:
Mod1E_results_final = pd.DataFrame(Mod1E_results_human[Mod1E_results_human['hit_symbol'].isin(high_counts)])

In [ ]:
Mod1E_results_final['module'] = 'Mod1E'

In [ ]:
Mod1E_results_final.head()

In [ ]:
all_results = pd.concat([Mod1A_results_human, Mod1B_results])

In [ ]:
from Modules.StandardOutput import StandardOutput

In [ ]:
so = StandardOutput(results=all_results.to_dict(orient='records'), input_object=input_object)

In [ ]:
std_api_response_json = so.output_object

In [ ]:
std_api_response_json

In [36]:
import requests

# get the URL for these results displayed in the RTX UI
RTX_UI_REQUEST_URL = "https://rtx.ncats.io/devED/api/rtx/v1/response/process"
to_post = {"options": ["Store", "ReturnResponseId"], "responses": [std_api_response_json]}
ui_url = requests.post(RTX_UI_REQUEST_URL, json=to_post)
print("Please visit the following website: https://rtx.ncats.io/?r=%s" % ui_url.json()['response_id'])

KeyError: 'response_id'

In [35]:
ui_url.json()['detail']

'The requested URL was not found on the server.  If you entered the URL manually please check your spelling and try again.'